# GROMACS CMDs

## Force field

In [ ]:
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/atomtypes.atp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/lipids.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/lipids.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.c.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.c.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.c.tdb

/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.hdb
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.rtp
/gromacs/AVX2_256_ts/share/gromacs/top/residuetypes.dat 
# /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/atomtypes.atp
# /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.n.tdb -> Seems like IONS
# /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.c.tdb -> Terminal ones, OXT

NLE 
residuetypes.dat <- 51: NLE Protein

8: CHARMM27 all-atom force field (CHARM22 plus CMAP for proteins)

1: TIP3P   TIP 3-point, recommended

# Equilibrium

In [1]:
import mdtraj as md
import nglview as nv

molecule="2f4k"
molecule_lower = molecule.lower()
molecule_upper = molecule.upper()

## Explicit TIP3P

In [2]:
cmds = []
## Initial system ##
cmds.append("Initial system configuration: folded.gro, topol.top")
cmds.append("gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm27 -water tip3p --ignh\n")
cmds.append("gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm36 -water tip3p --ignh\n")

cmds.append("Create box: folded_boxed.gro")
cmds.append("gmx editconf -f folded.gro -o folded_boxed.gro -c -d 1.2 -bt cubic -box 4.7 4.7 4.7\n")

cmds.append("Solvate: folded_solv.gro, topol.top")
cmds.append("gmx solvate -cp folded_boxed.gro -cs spc216.gro -o folded_solv.gro -p topol.top\n")

cmds.append("> Add ions: ions.tpr - WARNING on net charge")
cmds.append("gmx grompp -f ../common/ions.mdp -po ions.mdp -c folded_solv.gro -p topol.top -o ions.tpr -maxwarn 1")
cmds.append("> Select SOL (13): topol.top, folded_solv_ions.gro")
cmds.append("gmx genion -s ions.tpr -o folded_solv_ions.gro -p topol.top -pname NA -nname CL -neutral\n")



cmds.append("\n\nEnergy minimization (EM)")
cmds.append("> Energy minimization: em.tpr")
cmds.append("gmx grompp -f ../common/minim.mdp -c folded_solv_ions.gro -p topol.top -o em.tpr\n")

cmds.append("> Run EM MD: em.log, em.edr (energy), em.trr (full precision trajectory), em.gro (EM structure)")
cmds.append("gmx mdrun -v -deffnm em -ntomp 1 -pin on\n")

cmds.append("> Energy analysis (11 0): potential.xvg")
cmds.append("gmx energy -f em.edr -o potential.xvg\n")



cmds.append("\n\nEquilibration - temperature")
cmds.append("> Equilibration - temperature: nvt.tpr, nvt.gro, nvt.cpt, nvt.edr, nvt.log, nvt.trr, nvt.log, nvt.xtc")
cmds.append(f"gmx grompp -f ./{molecule_lower}_nvt.mdp -c em.gro -r em.gro -p topol.top -po {molecule_lower}_nvt_out.mdp -o nvt.tpr\n")

cmds.append("> Run NVT MD: nvt.log, nvt.xtc")
cmds.append("gmx mdrun -deffnm nvt -ntomp 1 -pin on -bonded gpu -nb gpu -pme gpu\n")

cmds.append("> Energy analysis (16 0): temperature.xvg")
cmds.append("gmx energy -f nvt.edr -o temperature.xvg\n")



cmds.append("\n\nEquilibration - pressure")
cmds.append("> Equilibration - pressure : npt.tpr, npt.gro, npt.cpt, npt.edr, npt.log, npt.trr, npt.log, npt.xtc")
cmds.append(f"gmx grompp -f ./{molecule_lower}_npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -po {molecule_lower}_npt_out.mdp -o npt.tpr\n")
cmds.append("> Run NPT MD: npt.log, npt.xtc")
cmds.append("gmx mdrun -deffnm npt -ntomp 1 -pin on -bonded gpu -nb gpu -pme gpu\n")

cmds.append("> Energy analysis (17 0): pressure.xvg")
cmds.append("gmx energy -f npt.edr -o pressure.xvg\n")

cmds.append("> Energy analysis (23 0): density.xvg")
cmds.append("gmx energy -f npt.edr -o density.xvg\n")



## Production MD ##
cmds.append("\n\n> Final tpr")
cmds.append(f"gmx grompp -f ./{molecule_lower}_md.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr")

for cmd in cmds:
    print(cmd)


Initial system configuration: folded.gro, topol.top
gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm27 -water tip3p --ignh

gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm36 -water tip3p --ignh

Create box: folded_boxed.gro
gmx editconf -f folded.gro -o folded_boxed.gro -c -d 1.2 -bt cubic -box 4.7 4.7 4.7

Solvate: folded_solv.gro, topol.top
gmx solvate -cp folded_boxed.gro -cs spc216.gro -o folded_solv.gro -p topol.top

> Add ions: ions.tpr - WARNING on net charge
gmx grompp -f ../common/ions.mdp -po ions.mdp -c folded_solv.gro -p topol.top -o ions.tpr -maxwarn 1
> Select SOL (13): topol.top, folded_solv_ions.gro
gmx genion -s ions.tpr -o folded_solv_ions.gro -p topol.top -pname NA -nname CL -neutral



Energy minimization (EM)
> Energy minimization: em.tpr
gmx grompp -f ../common/minim.mdp -c folded_solv_ions.gro -p topol.top -o em.tpr

> Run EM MD: em.log, em.edr (energy), em.trr (full precision trajectory), em.gro (EM structure)
gmx mdrun -v -deffnm 

## Visualization

In [ ]:
gro_path = f"./{molecule.upper()}/folded_solv_ions.gro"
gro_traj = md.load(gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget()

In [ ]:
xtc_path = f"./{molecule.upper()}/nvt.xtc"
gro_path = f"./{molecule.upper()}/nvt.gro"
gro_traj = md.load(xtc_path, top=gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget(max_frame=50)

In [ ]:
gro_path = f"./{molecule.upper()}/npt.gro"
gro_traj = md.load(gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget()

In [ ]:
gro_path = f"./{molecule.upper()}/folded.gro"
gro_traj = md.load(gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget()

In [ ]:
xtc_path = f"/home/shpark/prj-mlcv/lib/bioemu/opes/simulations/1fme/tica/debug/0.xtc"
gro_path = f"./{molecule.upper()}/folded.gro"
gro_traj = md.load_xtc(xtc_path, top=gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget(max_frame=20)